In [12]:
import pandas as pd


In [2]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [ ]:
ingest_create_athena_db_passed = False

## Import PyAthena

In [3]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

## Create Athena Database

In [26]:
database_name = "compens"

In [27]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [28]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [29]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS compens


In [30]:
pd.read_sql(statement, conn)

""


## Verify The Database Has Been Created Succesfully

In [31]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,compens
1,default
2,dsoaws


In [32]:
#Directory for the database
compens_dir = 's3://508-team4/data'

In [38]:
table_name1 ='la_compensation'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name1}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name1}(
                record_nbr string,
                pay_year int,
                department_no string,
                department_title string,
                job_class_pgrade int,
                job_title string,
                employment_type string,
                job_status string,
                mou string,
                mou_title string,
                regular_pay float,
                overtime_pay float,
                all_other_pay float,
                total_pay float,
                city_retirement_contributions float,
                benefit_pay float,
                gender string,
                ethnicity string
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION 's3://508-team4/data/'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

pd.read_sql(f'SELECT * FROM {database_name}.{table_name1} LIMIT 5', conn)
                

,record_nbr,pay_year,department_no,department_title,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,regular_pay,overtime_pay,all_other_pay,total_pay,city_retirement_contributions,benefit_pay,gender,ethnicity
0,333636333433,2016,82,PUBLIC WORKS - SANITATION,None,ENVIRONMENTAL COMPLIANCE INSPECTOR,FULL_TIME,ACTIVE,21.0,TECHNICAL,39353.60,0.00,134.68,39488.28,11672.28,3611.27,MALE,BLACK
1,3336363335,2016,6,ANIMAL SERVICES,None,ADMINISTRATIVE CLERK,FULL_TIME,ACTIVE,3.0,CLERICAL,52208.00,1027.85,0.00,53235.85,15484.89,14181.36,MALE,BLACK
2,333636333733,2016,44,LIBRARY,None,ADMINISTRATIVE CLERK,PART_TIME,NOT_ACTIVE,3.0,CLERICAL,10716.48,0.00,0.00,10716.48,0.00,0.00,FEMALE,ASIAN
3,333636343033,2016,94,TRANSPORTATION,None,CROSSING GUARD,PART_TIME,NOT_ACTIVE,34.0,CROSSING GUARDS,6762.81,0.00,3052.00,9814.81,0.00,0.00,MALE,HISPANIC
4,333636343233,2016,70,POLICE,None,CRIMINALIST II,FULL_TIME,NOT_ACTIVE,8.0,PROFESSIONAL ENGINEERING AND SCIENTIFIC,115342.40,11550.08,251.00,127143.48,34210.56,7350.00,MALE,ASIAN


In [39]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}